In [11]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser

In [12]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
filing_links = pd.read_excel(
    "/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC__sec_filing_links.xlsx")

path = '/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC_Investment.xlsx'
xls = pd.ExcelFile(path)
all_sheets = pd.read_excel(path, sheet_name=None)

In [13]:
dataframes = {}
# Loop through each sheet and create a DataFrame in the dictionary
for sheet_name, sheet_df in all_sheets.items():
    dataframes[sheet_name.replace(' ', '_')] = sheet_df
for sheet_name, sheet_df in dataframes.items():
    print(f"DataFrame name: {sheet_name} : {sheet_df.shape}")

DataFrame name: September_30_2010 : (800, 9)
DataFrame name: December_31_2010 : (1071, 12)
DataFrame name: March_31_2011 : (906, 9)
DataFrame name: June_30_2011 : (379, 7)
DataFrame name: September_30_2011 : (340, 7)
DataFrame name: December_31_2011 : (1181, 12)
DataFrame name: March_31_2012 : (2864, 7)
DataFrame name: June_30_2012 : (336, 7)
DataFrame name: September_30_2012 : (360, 7)
DataFrame name: December_31_2012 : (1075, 12)
DataFrame name: March_31_2013 : (714, 9)
DataFrame name: June_30_2013 : (1245, 13)
DataFrame name: September_30_2013 : (27, 7)
DataFrame name: December_31_2013 : (1100, 13)
DataFrame name: March_31_2014 : (1326, 13)
DataFrame name: June_30_2014 : (1354, 13)
DataFrame name: September_30_2014 : (1357, 12)
DataFrame name: December_31_2014 : (1190, 11)
DataFrame name: March_31_2015 : (1496, 13)
DataFrame name: June_30_2015 : (1542, 13)
DataFrame name: September_30_2015 : (1572, 13)
DataFrame name: December_31_2015 : (1379, 11)
DataFrame name: March_31_2016 : (20

In [14]:
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d %Y")
filing_links.head()

,Form type,Form description,Filing date,Reporting date,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],November 04 2010,September 30 2010,https://www.sec.gov/Archives/edgar/data/128775...
1,10-K,"Annual report [Section 13 and 15(d), not S-K I...",March 01 2011,December 31 2010,https://www.sec.gov/Archives/edgar/data/128775...
2,10-Q,Quarterly report [Sections 13 or 15(d)],May 03 2011,March 31 2011,https://www.sec.gov/Archives/edgar/data/128775...
3,10-Q,Quarterly report [Sections 13 or 15(d)],August 04 2011,June 30 2011,https://www.sec.gov/Archives/edgar/data/128775...
4,10-Q,Quarterly report [Sections 13 or 15(d)],November 08 2011,September 30 2011,https://www.sec.gov/Archives/edgar/data/128775...


In [18]:

matches = []  # To store rows of matching data

for file, frame in dataframes.items():
    if frame.shape[0] < 100:
        print(file.replace('_', ' '))
        matching_rows = filing_links[filing_links['Reporting date'].str.replace(
            ' ', '') == file.replace('_', '')]
        if not matching_rows.empty:
            matches.append(matching_rows)
            print(matching_rows['Filings URL'].values[0])
            # webbrowser.open(matching_rows['Filings URL'].values[0])
working_class = pd.concat(matches, ignore_index=True)

September 30 2013
https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm
March 31 2016
https://www.sec.gov/Archives/edgar/data/1287750/000110465916117420/a16-6593_110q.htm
June 30 2016
https://www.sec.gov/Archives/edgar/data/1287750/000110465916136353/a16-11519_110q.htm
September 30 2016
https://www.sec.gov/Archives/edgar/data/1287750/000110465916153937/a16-17135_110q.htm
March 31 2017
https://www.sec.gov/Archives/edgar/data/1287750/000162828017004707/arccq1-1710q.htm
June 30 2017
https://www.sec.gov/Archives/edgar/data/1287750/000128775017000012/arccq2-1710q.htm
September 30 2017
https://www.sec.gov/Archives/edgar/data/1287750/000128775017000017/arccq3-1710q.htm
December 31 2017
https://www.sec.gov/Archives/edgar/data/1287750/000128775018000007/arccq4-1710k.htm
March 31 2018
https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm
June 30 2018
https://www.sec.gov/Archives/edgar/data/1287750/000128775018000020/arccq2-1810q.htm
S